In [1]:
import os, re, sys, json, xmltodict, pandas
from io import StringIO
from collections import deque, OrderedDict
from Perdy.parser import doParse
from Spanners.Digger import Digger
from Spanners.Treeify import Treeify

In [2]:
df = pandas.read_excel('data.xlsx').astype(str)
digger = Digger(dq=df, stack=[OrderedDict()])

In [3]:
digger.dq

,A,B,C
0,1,2,4
1,1,2,5
2,1,3,6
3,1,3,7


In [4]:
digger.dig()

1
	2
		4
		5
	3
		6
		7


In [5]:
digger.stack

[OrderedDict([('outline',
               [{'@text': '1',
                 'outline': [{'@text': '2',
                   'outline': [{'@text': '4'}, {'@text': '5'}]},
                  {'@text': '3',
                   'outline': [{'@text': '6'}, {'@text': '7'}]}]}])])]

In [6]:
treeify = Treeify(
    ascii = False,
    colour = True,
)

treeify.process(StringIO(json.dumps(digger.stack)), sys.stdout)

[0]
 └─outline
    └─[0]
       ├─@text
       │  └─1
       └─outline
          ├─[0]
          │  ├─@text
          │  │  └─2
          │  └─outline
          │     ├─[0]
          │     │  └─@text
          │     │     └─4
          │     └─[1]
          │        └─@text
          │           └─5
          └─[1]
             ├─@text
             │  └─3
             └─outline
                ├─[0]
                │  └─@text
                │     └─6
                └─[1]
                   └─@text
                      └─7